## Deep Reinforcement Learning using AI Gym's Frozen Lake

This is an example of using deep reinforcement learning to determine the best policy for navigating the AI gym's Frozen Lake problem.
Traditionally this is solved using MDPs, but it can be solved model free using deep learning (as you would expect)

I was inspired by Miguel Morales book on Deep Reinforcement Learning (which I hope he finishes soon)

It uses a double Q deep learning structure, where it only updates the model after running a number of games (rather than just one). This provides stability so one particular game does not dominate the learning.

This should be used in conjuction with reading about deep reinforcement learning and really this just serves as a functional code.

In [0]:
#@title MIT License
#
# Copyright (c) 2019 Graham Anthony Chow
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

This example was written and tested in Tensor Flow 2.0 beta

In [0]:
# Install TensorFlow
!pip install tensorflow==2.0.0-beta1

import gym
import numpy as np
import tensorflow as tf


Let get the Frozen Lake....

In [0]:
#get the game
env = gym.make('FrozenLake-v0')

#labeling text to mapping
action_map = {
    0 : "left",
    1 : "down",
    2 : "right",
    3 : "up "}

#result expected, one that you would get from using MDPs to solve this.
ideal_mdp_result = [ "(left) ", "(up)   ", "(left) ", "(up)   ", "(left) ", "(left) ", "(up)   ", "(down) ", "(left) ", "(right)", "(down) "]

Since we are only 11 locations that will have a policy, lets create a map these away so the network does not need to worry about them.

In [0]:
#even though the grid is 4 x 4, only 11 locations are valid (the goal and pits are don't have really any policy to learn) , so lets just learn on those ones
state_map = {
    0 : 0,
    1 : 1,
    2 : 2,
    3 : 3,
    4 : 4,
    5 : 6,
    6 : 8,
    7 : 9,
    8 : 10,
    9 : 13,
    10 : 14
    }

reverse_state_map = {
    0 : 0,
    1 : 1,
    2 : 2,
    3 : 3,
    4 : 4,
    5 : -1,
    6 : 5,
    7 : -1,
    8 : 6,
    9 : 7,
    10 : 8,
    11 : -1,
    12 : -1,
    13 : 9,
    14 : 10,
    15 : -1
    }

# not mapped locations
non_mapped = [ 5, 7, 11, 12, 15]

Now lets define this network. You can try to change to see what happens.

In [0]:
#set up the network, a random network ;)
l0 = tf.keras.layers.Dense(units=11, input_shape=[11]) 
l1 = tf.keras.layers.Dense(units=44, activation='relu') 
l2 = tf.keras.layers.Dense(units=44, activation='relu') 
l3 = tf.keras.layers.Dense(units=4)
model = tf.keras.Sequential([l0, l1, l2, l3])
model.compile(loss='mean_squared_error',
            optimizer=tf.keras.optimizers.Adam(0.001))
print(model.summary())

Lets play a few games to see what happens if you randomly decide where to go

In [0]:
# test on an untrained network
for t in range(10):
    s = env.reset()

    j = 0
    while j < 999: #just make sure we end, however very unlikely we would just go around circles forever
        j+=1

        sm = reverse_state_map[s]
        input = np.identity(11)[sm:sm+1]
        predictions = model.predict(input)
        a = np.argmax(predictions[0])

        #perform the step
        s1,r,d,_ = env.step(a)    

        if d == True:
            if r > 0:
                print(f"Game{t}: success, you reached the goal")
            else:
                print(f"Game{t}: fail, you fell in a pit")
            break;
        s = s1

That did not work out so well.


Lets do some training to explore to determine where to go.

In [0]:
#training
epsilon = 1
num_episodes = 1000
epsilon_decay_rate = 1/num_episodes

# loss for each step away from goal
gamma = 0.9;

for t in range(num_episodes):
    x = []
    y = []
    #this is double Q, so we run the game 100 times before we update the network
    for b in range(100):
        #reset the environment
        s = env.reset()

        j = 0
        while j < 999: #just make sure we end, however very unlikely we would just go around circles forever
            j+=1

            #predict which way we should go according to the network (initially this will be random)
            sm = reverse_state_map[s]
            assert(sm >= 0)
            input = np.identity(11)[sm:sm+1]
            predictions = model.predict(input)

            # intially we need to explore the map and hence the random steps. However we need to progressively actually get some where so the rewards can filter back and we determine the correct policy.
            if np.random.rand(1) < epsilon:
                a = env.action_space.sample()
            else:
                a = np.argmax(predictions[0])
            
            #perform the step
            s1,r,d,_ = env.step(a)
            if d == True:
                update = r #either we have reached the goal or fallen down a pit
            else:
                s1m = reverse_state_map[s1]
                assert(s1m >= 0)
                input1 = np.identity(11)[s1m:s1m+1]
                predictions1 = model.predict(input1) 
                update = r + gamma * np.max(predictions1[0])

            #we only update the action we take, all the others we don't try to change
            predictions[0][a] = update

            # store for training
            y.append(predictions[0])
            x.append(input[0])

            s = s1
            if d == True:
                break;

    #now do some training together with some UI
    if t % 25 == 0:
        print(f"{t} {epsilon}")
        model.fit(np.array(x), np.array(y), verbose=True)
        for s in range(16):
            if reverse_state_map[s] < 0:
                print(f"{s}")
            else:
                sm = reverse_state_map[s]
                input = np.identity(11)[sm:sm+1]
                predictions = model.predict(input)
                print(f"{s} {predictions} {ideal_mdp_result[sm]} {action_map[np.argmax(predictions[0])]}")
    else:
        model.fit(np.array(x), np.array(y), verbose=False)

    epsilon -= epsilon_decay_rate
    
print(f"Finished!")    

Lets look at the final policy

In [0]:
for s in range(16):
    if reverse_state_map[s] < 0:
        print(f"{s}")
    else:
        sm = reverse_state_map[s]
        input = np.identity(11)[sm:sm+1]
        predictions = model.predict(input)
        print(f"{s} {predictions} {ideal_mdp_result[sm]} {action_map[np.argmax(predictions[0])]}")

Network fully trained, now lets try those games again

In [0]:
for t in range(10):
    s = env.reset()

    j = 0
    while j < 999: #just make sure we end, however very unlikely we would just go around circles forever
        j+=1

        sm = reverse_state_map[s]
        input = np.identity(11)[sm:sm+1]
        predictions = model.predict(input)
        a = np.argmax(predictions[0])

        #perform the step
        s1,r,d,_ = env.step(a)    

        if d == True:
            if r > 0:
                print(f"Game{t}: success, you reached the goal")
            else:
                print(f"Game{t}: fail, you fell in a pit")
            break;
        s = s1